Calculate taus:

on unit level:
1. Pearsonr trial average
2. STTC trial average
3. STTC trial concat
   
on trial level (not calculated yet):
1. Pearsonr per trial
2. ACF proper per trial
3. iSTTC per trial

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from isttc.scripts.cfg_global import project_folder_path
from isttc.tau import fit_single_exp, func_single_exp_monkey

In [2]:
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rcParams['svg.fonttype'] = 'none'

In [3]:
empty_suffix = 'no'
data_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms_' + empty_suffix + '_empty\\'
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms_' + empty_suffix + '_empty\\'

### Functions

In [4]:
def calc_tau_per_unit(acf_df_, acf_cols_, start_idx_):
    """
    Calculate tau values per unit using single exponential fitting.

    Parameters:
    - acf_df (pd.DataFrame): DataFrame containing autocorrelation function (ACF) values.
    - acf_cols (list): List of column names containing ACF data.
    - start_idx (int): Starting index for fitting.

    Returns:
    - pd.DataFrame: DataFrame with fitting parameters and tau values per unit.
    """
    acf_2d = acf_df_[acf_cols_].values.astype(np.float64)
    n_units = acf_2d.shape[0]
    print(f'Calculating taus for {acf_2d.shape}')
    
    results = [fit_single_exp(acf_2d[i, :], start_idx_, func_single_exp_monkey) for i in range(n_units)]

    fit_popt_a, fit_popt_b, fit_popt_c, fit_tau, tau_ci_lower, tau_ci_upper, fit_r_squared, explained_var, fit_log_message = zip(*[
        (fit_popt[0] if isinstance(fit_popt, np.ndarray) else np.nan,
         fit_popt[1] if isinstance(fit_popt, np.ndarray) else np.nan,
         fit_popt[2] if isinstance(fit_popt, np.ndarray) else np.nan,
         tau, tau_ci[0], tau_ci[1], fit_r_squared, explained_var, log_msg)
        for fit_popt, _, tau, tau_ci, fit_r_squared, explained_var, log_msg in results
    ])

    # fit_popt, fit_pcov, tau, tau_ci, fit_r_squared, explained_var, log_message

    tau_df = pd.DataFrame({
        'unit_id': acf_df_['unit_id'].values,
        'fit_a': fit_popt_a,
        'fit_b': fit_popt_b,
        'fit_c': fit_popt_c,
        'tau': fit_tau,
        'tau_ci_lower': tau_ci_lower,
        'tau_ci_upper': tau_ci_upper,
        'r_squared': fit_r_squared,
        'explained_var': explained_var,
        'log_message': fit_log_message
    })

    float_cols = ['fit_a', 'fit_b', 'fit_c', 'tau', 'r_squared','explained_var', 'tau_ci_lower', 'tau_ci_upper']
    tau_df[float_cols] = tau_df[float_cols].astype(float)
    
    return tau_df

### Calculate tau

#### One tau per unit

3 methods of doing that:
* Pearsonr trial average (as in the paper)
* STTC trial average
* STTC trial concat

todo: take ACF average over trails for a unit and fit? Or fit all ACFs with one function? (this requires calculating ACF per trial)

In [5]:
n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


In [ ]:
area = 'pfdl' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_lower_ms'] = tau_pearsonr_trial_avg_df['tau_ci_lower'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_upper_ms'] = tau_pearsonr_trial_avg_df['tau_ci_upper'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df['tau_ci_lower_ms'] = tau_sttc_trial_avg_df['tau_ci_lower'] * bin_size
tau_sttc_trial_avg_df['tau_ci_upper_ms'] = tau_sttc_trial_avg_df['tau_ci_upper'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df['tau_ci_lower_ms'] = tau_sttc_trial_concat_df['tau_ci_lower'] * bin_size
tau_sttc_trial_concat_df['tau_ci_upper_ms'] = tau_sttc_trial_concat_df['tau_ci_upper'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

In [ ]:
tau_sttc_trial_avg_df

In [ ]:
tau_pearsonr_trial_avg_df

In [8]:
area = 'pfp' 

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_lower_ms'] = tau_pearsonr_trial_avg_df['tau_ci_lower'] * bin_size
tau_pearsonr_trial_avg_df['tau_ci_upper_ms'] = tau_pearsonr_trial_avg_df['tau_ci_upper'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df['tau_ci_lower_ms'] = tau_sttc_trial_avg_df['tau_ci_lower'] * bin_size
tau_sttc_trial_avg_df['tau_ci_upper_ms'] = tau_sttc_trial_avg_df['tau_ci_upper'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df['tau_ci_lower_ms'] = tau_sttc_trial_concat_df['tau_ci_lower'] * bin_size
tau_sttc_trial_concat_df['tau_ci_upper_ms'] = tau_sttc_trial_concat_df['tau_ci_upper'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_' + empty_suffix + '_empty_50ms_20lags_df.pkl')

Calculating taus for (538, 20)
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf contains NaNs, low spike count
ValueError: array must not contain infs or NaNs
Possible reason: acf 

In [ ]:
#### Calculate unit 

#### One tau per trial

3 methods of doing that:
* Pearsonr
* ACF formula
* iSTTC

In [ ]:
# n_lags = 20
# bin_size = 50
# acf_cols = ['acf_' + str(i) for i in range(n_lags)]
# print('acf_cols {}'.format(acf_cols))

In [ ]:
# area = 'pfdl' 

# pearsonr 
# pearsonr_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(pearsonr_per_trial_df)}, dropping before tau calc ...')
# pearsonr_per_trial_df.dropna(inplace=True)
# pearsonr_per_trial_df.reset_index(drop=True, inplace=True)
# tau_pearsonr_per_trial_df = calc_tau_per_unit(pearsonr_per_trial_df, acf_cols[:-1], start_idx_=1)
# tau_pearsonr_per_trial_df['tau_ms'] = tau_pearsonr_per_trial_df['tau'] * bin_size
# tau_pearsonr_per_trial_df.insert(1, 'trial_id', pearsonr_per_trial_df['trial_id'])
# tau_pearsonr_per_trial_df.insert(2, 'condition_id', pearsonr_per_trial_df['condition_id'])
# tau_pearsonr_per_trial_df.insert(3, 'spike_count', pearsonr_per_trial_df['spike_count'])
# tau_pearsonr_per_trial_df.insert(4, 'fr_hz', pearsonr_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_pearsonr_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_pearsonr_per_trial_df)}')
# tau_pearsonr_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# # acf formula
# pcf_proper_per_trial_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {pcf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(pcf_proper_per_trial_df)}, dropping before tau calc ...')
# pcf_proper_per_trial_df.dropna(inplace=True)
# pcf_proper_per_trial_df.reset_index(drop=True, inplace=True)
# tau_acf_proper_per_trial_df = calc_tau_per_unit(pcf_proper_per_trial_df, acf_cols, start_idx_=1)
# tau_acf_proper_per_trial_df['tau_ms'] = tau_acf_proper_per_trial_df['tau'] * bin_size
# tau_acf_proper_per_trial_df.insert(1, 'trial_id', pcf_proper_per_trial_df['trial_id'])
# tau_acf_proper_per_trial_df.insert(2, 'condition_id', pcf_proper_per_trial_df['condition_id'])
# tau_acf_proper_per_trial_df.insert(3, 'spike_count', pcf_proper_per_trial_df['spike_count'])
# tau_acf_proper_per_trial_df.insert(4, 'fr_hz', pcf_proper_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_acf_proper_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_acf_proper_per_trial_df)}')
# tau_acf_proper_per_trial_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_acf_proper_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')

# # isttc
# sttc_per_trial_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')
# print(f'N rows with NaN {sttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(sttc_per_trial_df)}, dropping before tau calc ...')
# sttc_per_trial_df.dropna(inplace=True)
# sttc_per_trial_df.reset_index(drop=True, inplace=True)
# tau_isttc_per_trial_df = calc_tau_per_unit(sttc_per_trial_df, acf_cols, start_idx_=1)
# tau_isttc_per_trial_df['tau_ms'] = tau_isttc_per_trial_df['tau'] * bin_size
# tau_isttc_per_trial_df.insert(1, 'trial_id', sttc_per_trial_df['trial_id'])
# tau_isttc_per_trial_df.insert(2, 'condition_id', sttc_per_trial_df['condition_id'])
# tau_isttc_per_trial_df.insert(3, 'spike_count', sttc_per_trial_df['spike_count'])
# tau_isttc_per_trial_df.insert(4, 'fr_hz', sttc_per_trial_df['fr_hz'])
# print(f'N rows with NaN {tau_isttc_per_trial_df.isnull().any(axis=1).sum()} out of {len(tau_isttc_per_trial_df)}')
# tau_isttc_per_trial_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_isttc_per_trial_1000ms_with_empty_50ms_20lags_df.pkl')